In [ ]:
!pip install -U --user numpy

In [ ]:
!pip install wget

In [1]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import urllib
import wget

MAX_LEN = 512
EPOCHS = 3
VERBOSE = 2
BATCH_SIZE = 16

In [2]:
DATA_OUT_PATH = './data_out/KOR'

In [3]:
def plot_graphs(history, string, string_1, string_2):
    # loss 
    plt.plot(history.history[string])
    plt.plot(history.history[string_1])
    plt.plot(history.history[string_2])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, string_1, string_2])
    plt.show()

In [4]:
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)
np.random.seed(SEED_NUM)

In [5]:
# Save the slow pretrained tokenizer
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", lowercase=False)
save_path = "bert-base-multilingual-cased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("bert-base-multilingual-cased/vocab.txt", lowercase=False)

In [14]:
train_data_url = "https://korquad.github.io/dataset/KorQuAD_v1.0_train.json"
train_path = keras.utils.get_file("train.json", train_data_url)
eval_data_url = "https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json"
eval_path = keras.utils.get_file("eval.json", eval_data_url)

38543360/38527475 [==============================] - 1s 0us/step


In [7]:
wget.download('https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json', out='./bert-base-multilingual-cased/')

100% [.......................................................] 625 / 625

'./bert-base-multilingual-cased//bert-base-multilingual-cased-config (1).json'

In [9]:
os.rename('./bert-base-multilingual-cased/bert-base-multilingual-cased-config.json', './bert-base-multilingual-cased/config.json')

In [10]:
wget.download('https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-tf_model.h5', out='./bert-base-multilingual-cased/')

100% [.........................................] 1083389348 / 1083389348

'./bert-base-multilingual-cased//bert-base-multilingual-cased-tf_model.h5'

In [11]:
os.rename('./bert-base-multilingual-cased/bert-base-multilingual-cased-tf_model.h5', './bert-base-multilingual-cased/tf_model.h5')

In [28]:
a = """밑바닥     부터    시작하는 


딥러닝"""
print(str(a).split())
a = " ".join(str(a).split())
print(a)

['밑바닥', '부터', '시작하는', '딥러닝']
밑바닥 부터 시작하는 딥러닝


In [71]:
class SquadExample:
    def __init__(self, question, context, start_char_idx, answer_text):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx

#         print('context=',context)
#         print('question=',question)
#         print('answer_text=',answer_text)
#         print('start_char_idx=',start_char_idx)
        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())

        # Find end character index of answer in context
        end_char_idx = start_char_idx + len(answer)
#         print('end_char_idx=',end_char_idx)
        if end_char_idx >= len(context):
            self.skip = True
            return

        # Mark the character indexes in context that are in answer
        is_char_in_ans = [0] * len(context)
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1
        
#         print("is_char_in_ans = ", is_char_in_ans)

        # Tokenize context
        tokenized_context = tokenizer.encode(context)

# [ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]        
#         print("tokenized_context = ", tokenized_context)
#         print('tokenized_context.ids=', tokenized_context.ids)
#         print('tokenized_context.type_ids=', tokenized_context.type_ids)
#         print('tokenized_context.tokens=', tokenized_context.tokens)
#         print('tokenized_context.offsets=',tokenized_context.offsets)
#         print('tokenized_context.attention_mask=', tokenized_context.attention_mask)
#         print('tokenized_context.special_tokens_mask=', tokenized_context.special_tokens_mask)
        
        # Find tokens that were created from answer characters
        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_context.offsets):
            if sum(is_char_in_ans[start:end]) > 0:
                ans_token_idx.append(idx)
                
#         print("ans_token_idx=", ans_token_idx)

        if len(ans_token_idx) == 0:
            self.skip = True
            return

        # Find start and end token index for tokens from answer
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]
        
#         print('start_token_idx=', start_token_idx)
#         print('end_token_idx=', end_token_idx)

        # Tokenize question
        tokenized_question = tokenizer.encode(question)

        # Create inputs

#         print('tokenized_question.ids=', tokenized_question.ids)
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
#         print('input_ids=', input_ids)
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
#         print('token_type_ids=', token_type_ids)
        attention_mask = [1] * len(input_ids)
#         print('attention_mask=', attention_mask)

#         print("len(input_ids)=", len(input_ids))
        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = MAX_LEN - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

#         print('input_ids=', input_ids)
#         print('token_type_ids=', token_type_ids)
#         print('attention_mask=', attention_mask)
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_context.offsets


def create_squad_examples(raw_data):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
#             print('context = ' , context)
            for qa in para["qas"]:
                question = qa["question"]
#                 print('question = ' , question)
                
                answer_text = qa["answers"][0]["text"]
#                 print('answer_text = ', answer_text)
                start_char_idx = qa["answers"][0]["answer_start"]
#                 print('start_char_idx = ', start_char_idx)
                squad_eg = SquadExample(
                    question, context, start_char_idx, answer_text
                )
                squad_eg.preprocess()
                squad_examples.append(squad_eg)
#                 break
#             break
#         break
    return squad_examples


def create_inputs_targets(squad_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
#             print(dataset_dict)
#             break
#         break
            
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y


In [67]:
print(train_path)

C:\Users\jikim\.keras\datasets\train.json


In [68]:
with open(train_path) as f:
    raw_train_data = json.load(f)

print(type(raw_train_data))    
    
with open(eval_path) as f:
    raw_eval_data = json.load(f)

#         self.input_ids = input_ids
#         self.token_type_ids = token_type_ids
#         self.attention_mask = attention_mask
#         self.start_token_idx = start_token_idx
#         self.end_token_idx = end_token_idx
#         self.context_token_to_char = tokenized_context.offsets

train_squad_examples = create_squad_examples(raw_train_data)
# print(len(train_squad_examples))
# print(train_squad_examples[0].input_ids)
# print(train_squad_examples[0].token_type_ids)
# print(train_squad_examples[0].attention_mask)
# print(train_squad_examples[0].start_token_idx)
# print(train_squad_examples[0].end_token_idx)
# print(train_squad_examples[0].context_token_to_char)


<class 'dict'>
60407
[101, 16221, 10954, 9318, 78136, 70162, 11018, 8905, 119351, 10459, 9901, 89108, 15184, 10622, 62849, 9642, 11664, 8924, 8996, 24974, 10530, 9246, 32158, 10739, 8973, 26737, 35756, 9448, 36210, 11261, 9960, 12424, 90387, 8907, 79544, 55670, 10622, 9511, 26737, 11018, 9153, 10622, 8854, 40410, 119, 9638, 9485, 12310, 9318, 78136, 70162, 11018, 16347, 27056, 9387, 9088, 119267, 11467, 9407, 16617, 15891, 54918, 9056, 100, 9414, 65649, 10739, 17342, 9686, 58931, 11882, 9489, 89292, 10530, 8843, 118813, 51491, 9272, 97146, 12605, 26444, 119394, 100929, 11513, 9248, 49742, 9901, 89108, 15184, 10459, 9491, 31720, 10530, 8896, 105197, 12490, 11664, 16139, 119, 19789, 9901, 46766, 12424, 9519, 52015, 77884, 20308, 10459, 9706, 119455, 11261, 9901, 12692, 74293, 14279, 8900, 30842, 119110, 24989, 10739, 9568, 16323, 12178, 9344, 26444, 118979, 10459, 8907, 79544, 55670, 130, 35465, 10622, 9116, 11664, 8938, 10892, 8848, 52859, 9322, 119118, 41850, 117, 9638, 97403, 9638, 11

In [132]:
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")
# print(x_train[0][0])

eval_squad_examples = create_squad_examples(raw_eval_data)
x_eval, y_eval = create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")

60407 training points created.
5774 evaluation points created.


In [146]:
idx = 1

# print(x_train[0][0])
context = tokenizer.decode(x_train[0][idx])
print(context)

tokenized_context = tokenizer.encode(context)
offset = tokenized_context.offsets
print(offset)

start = y_train[0][idx]
end = y_train[1][idx]
print(start, end)

start = offset[start][0]
end = offset[end][1]

print(start, end)
print(context[start:end])


1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡 ( 1악장 ) 을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다. 바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?
[(0, 0), (0, 4), (4, 5), (6, 7), (7, 8), (8, 9), (9, 10), (11, 12), (12, 13), (13, 14), (15, 16), (16, 18), (18, 19), (19, 20), (21, 23), (24, 25), (25, 26), (27, 28), (29, 30), (30, 31), (31, 32), (33, 34), (34, 35), (35, 36), (37, 38), (38, 39), (40, 42), (43, 44), (44, 45), (45, 46), (47, 48

In [94]:
class TFBERTQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBERTQuestionAnswering, self).__init__()
        
        self.encoder = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.start_logit = tf.keras.layers.Dense(num_class, name="start_logit", use_bias=False) #(N,T,D)(D,1)=(N,T,1)
        self.end_logit = tf.keras.layers.Dense(num_class, name="end_logit", use_bias=False)
        self.flatten = tf.keras.layers.Flatten() 
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)
        
    def call(self, inputs):
        input_ids, token_type_ids, attention_mask = inputs
        embedding = self.encoder(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
#         print(embedding)
#         print(embedding.last_hidden_state)
#         print(embedding[0])
        embedding = embedding[0]
        print(embedding.shape)   # (1,512,768)
        start_logits = self.start_logit(embedding)
        print(start_logits.shape) # (1,512,1)
        start_logits = self.flatten(start_logits)
        print(start_logits.shape) # (1,512)
        
        end_logits = self.end_logit(embedding)
        end_logits = self.flatten(end_logits)
        
        start_probs = self.softmax(start_logits)
        end_probs = self.softmax(end_logits)
    
        return start_probs, end_probs

In [95]:
korquad_model = TFBERTQuestionAnswering(model_name='./bert-base-multilingual-cased/',dir_path='bert_ckpt', num_class=1)
inputs = [x_train[0][0:1] , x_train[1][0:1], x_train[2][0:1]]
print(inputs[0].shape)
start_probs, end_probs = korquad_model(inputs)
print(np.argmax(start_probs.numpy))
print(np.argmax(end_probs.numpy))

Some layers from the model checkpoint at ./bert-base-multilingual-cased/ were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at ./bert-base-multilingual-cased/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


(1, 512)
(1, 512, 768)
(1, 512, 1)
(1, 512)
0
0


In [96]:
korquad_model = TFBERTQuestionAnswering(model_name='./bert-base-multilingual-cased/',dir_path='bert_ckpt', num_class=1)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)

Some layers from the model checkpoint at ./bert-base-multilingual-cased/ were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at ./bert-base-multilingual-cased/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [97]:
def normalized_answer(s):    
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text) 
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)   
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)      
        return text

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

In [98]:
class ExactMatch(keras.callbacks.Callback):
    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        count = 0
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            squad_eg = eval_examples_no_skip[idx]
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = squad_eg.context[pred_char_start:]

            normalized_pred_ans = normalized_answer(pred_ans)
            normalized_true_ans = normalized_answer(squad_eg.answer_text)
            if normalized_pred_ans in normalized_true_ans:
                count += 1
        acc = count / len(self.y_eval[0])
        print(f"\nepoch={epoch+1}, exact match score={acc:.2f}")

In [99]:
exact_match_callback = ExactMatch(x_eval, y_eval)

In [100]:
korquad_model.compile(optimizer=optimizer, loss=[loss, loss])

In [101]:
model_name = "tf2_bert_korquad"

checkpoint_path = os.path.join(DATA_OUT_PATH, model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(
    checkpoint_path, verbose=1, save_best_only=True, save_weights_only=True)

./data_out/KOR\tf2_bert_korquad -- Folder already exists 



In [102]:
history = korquad_model.fit(
    x_train,
    y_train,
    epochs=EPOCHS,  # For demonstration, 3 epochs are recommended
    verbose=VERBOSE,
    batch_size=BATCH_SIZE,
    callbacks=[exact_match_callback, cp_callback]
)

Epoch 1/3
(None, 512, 768)
(None, 512, 1)
(None, 512)
(None, 512, 768)
(None, 512, 1)
(None, 512)


KeyboardInterrupt: 

In [ ]:
print(history.history)

In [ ]:
plot_graphs(history, 'loss', 'output_1_loss', 'output_2_loss')

In [ ]:
korquad_model.save_weights('data_out/KOR/tf2_bert_korquad/weights.h5')

In [104]:
korquad_model.load_weights('data_out/KOR/tf2_bert_korquad/weights.h5')

In [ ]:
korquad_model.loss

In [ ]:
plot_graphs(korquad_model.history,'loss', 'output_1_loss', 'output_2_loss')

### 모델 predict 구현

In [105]:
korquad_model.load_weights('data_out/KOR/tf2_bert_korquad/weights.h5')

#### 지문 찾아보기

In [135]:
idx = 20
print(tokenizer.decode(x_eval[0][idx]))

노터데임 대학교에서 2년간 합리적으로 심각한 공부를 한 후 헤이그는 1944년 미국 육군사관학교로 임명을 획득하여 자신의 어린 시절을 군사 경력의 야망으로. 그 경력은 헤이그의 학문적 경연이 암시하려고 한것보다 더욱 극적이었으며 그는 1947년 310의 동기병에서 217번째 사관으로서 졸업하였다. 22세의 소위로 헤이그는 처음에 캔자스 주 포트라일리에서 정통 제병 연합부대로, 그러고나서 켄터키 주 포트녹스에 있는 기갑 훈련소로 갔다. 그후에 그는 제1 기병 사단으로 선임되고 그러고나서 일본에서 점령군의 임무와 기력이 없는 훈련을 하였다. 그는 1950년 5월 한번 자신의 사령관 알론조 폭스 장군의 딸 퍼트리샤 앤토이넷 폭스와 결혼하여 슬하 3명의 자식을 두었다. 알렉산더 헤이그가 미국 육군사관학교로 임명받은 해는 언제인가?


In [136]:
print(x_eval[0][idx])
print(x_eval[1][idx])
print(x_eval[2][idx])

[   101   9022  21876  28911  36240  72085  11489  92360  18784   9957
  12692  17022   9491  66540  11102   8896  73969   9954  10003   9973
  10739  78136  11018  10825  10954  23545   9626  17360  12945  20595
  46599  11261   9644  52859   9999 118813  13374  31956 106320   9485
  58931  10622   8910  12945   8885  28143  10459   9538  89292  11467
    100    119   8924   8885  28143  10892   9973  10739  78136  10459
   9953  25934  14801   8885  25486  10739   9526  14040  35506  26737
  11664   9954 118627  80001  99958   8925  14801  10739  52476  17889
  11113  10954  23993  10459   9095  12310  73380  11489  21651  48506
   9405  20595  11467  12424  45004  12609    119  10306  65443   9448
  92454   9973  10739  78136  11018  62849  10530   9793  13764  12605
   9689   9928  15184  17342  18392  46766  12424   9670  43022   9672
  73380   9568  33188  14646  37601    117   8924  30873  11664  16439
  12424   9807  21876  21039   9689   9928  15184 118742  12605  10530
  1376

In [110]:
tokenized_context= tokenizer.encode("법무부 장관을 제쳐놓고 민정수석이 개정안을 설명하는 게 이해가 안 된다고 지적한 경희대 석좌교수 이름은?")

In [111]:
context_token_to_char = tokenized_context.offsets
print(len(context_token_to_char))
print(context_token_to_char)

43
[(0, 0), (0, 1), (1, 2), (2, 3), (4, 5), (5, 6), (6, 7), (8, 9), (9, 10), (10, 11), (11, 12), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (19, 20), (20, 21), (21, 22), (22, 23), (24, 25), (25, 26), (26, 28), (29, 30), (31, 32), (32, 33), (33, 34), (35, 36), (37, 39), (39, 40), (41, 42), (42, 43), (43, 44), (45, 46), (46, 47), (47, 48), (49, 50), (50, 51), (51, 52), (52, 53), (54, 57), (57, 58), (0, 0)]


In [112]:
for i in range(len(context_token_to_char)):
    print(tokenizer.decode([tokenized_context.ids[i]]), end=" ")

 법 ##무 ##부 장 ##관 ##을 제 ##쳐 ##놓 ##고 민 ##정 ##수 ##석 ##이 개 ##정 ##안 ##을 설 ##명 ##하는 게 이 ##해 ##가 안 된다 ##고 지 ##적 ##한 경 ##희 ##대 석 ##좌 ##교 ##수 이름은 ?  

In [143]:
class SquadExample_pred:
    def __init__(self, question, context):#, start_char_idx, answer_text):
        self.question = question
        self.context = context
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        #answer_text = self.answer_text
        #start_char_idx = self.start_char_idx

#         print('context=',context)
#         print('question=',question)
        
        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        
#         print('context=',context)
#         print('question=',question)

        # Tokenize context
        tokenized_context = tokenizer.encode(context)
#         print("tokenized_context.ids=", tokenized_context.ids)
#         print("tokenized_context.ids.len=", len(tokenized_context.ids))

        # Tokenize question
        tokenized_question = tokenizer.encode(question)
#         print("tokenized_question.ids=", tokenized_question.ids)
        print(tokenized_context.tokens)

        # Create inputs
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
#         print("input_ids=", input_ids)
        
        
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
#         print("token_type_ids=", token_type_ids)
        
        attention_mask = [1] * len(input_ids)
#         print("attention_mask=", attention_mask)
        

        # Pad and create attention masks.
        # Skip if truncation is needed
        
        padding_length = MAX_LEN - len(input_ids)
        print("padding_length=",padding_length)
        
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return
        
#         print("input_ids=", input_ids)
#         print("token_type_ids=", token_type_ids)
#         print("attention_mask=", attention_mask)

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.context_token_to_char = tokenized_context.offsets
#         print(self.context_token_to_char)
        
    def get_input_target(self):
        dataset_dict = {
            "input_ids": [],
            "token_type_ids": [],
            "attention_mask": [],
        }
        if self.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(self, key))
        for key in dataset_dict:
            dataset_dict[key] = np.array(dataset_dict[key])

        x = [
            dataset_dict["input_ids"],
            dataset_dict["token_type_ids"],
            dataset_dict["attention_mask"],
        ]
#         print(x[0].shape)
        return x

def create_squad_examples_from_arg(question, context):#, start_char_idx, answer_text):
    squad_eg = SquadExample_pred(
        question, context#, start_char_idx, answer_text
    )
    squad_eg.preprocess()
    return squad_eg


def predict_test(model, pred_raw):
    x_pred = pred_raw.get_input_target()
    pred_start, pred_end = model.predict(x_pred)
#     print(pred_start.shape)
#     print(pred_end.shape)
#     print(pred_start)
    pred_start_offset_index = np.argmax(pred_start)
    pred_end_offset_index = np.argmax(pred_end)
    print(pred_start_offset_index, pred_end_offset_index)
#     print(pred_raw.context_token_to_char)
    pred_start_offset = pred_raw.context_token_to_char[pred_start_offset_index]
    pred_end_offset = pred_raw.context_token_to_char[pred_end_offset_index]
    print(pred_start_offset, pred_end_offset)

    answer = pred_context[pred_start_offset[0]:pred_end_offset[1]]
    
    normalized_pred_ans = normalized_answer(answer)
    
    return normalized_pred_ans


pred_context = '''정부가 이산화탄소 스트림의 수출을 가능하게 하는 런던의정서개정(2009년 개정) 수락서를 국제해사기구(IMO) 사무국에 기탁할 예정이라고 29일 밝혔다. 이산화탄소 스트림은 제철소나 발전소 등에서 포집한 이산화탄소다.
1996년 채택된 런던의정서는 자국 해역에서 이산화탄소 스트림 격리(저장)는 허용하는 반면, 이산화탄소 스트림 수출(국가 간 이동)은 금지했다. 
2009년 당사국총회에서 일정 절차에 따라 이산화탄소 스트림 수출을 허용하는 개정안이 채택됐다. '''
pred_question = "이산화탄소 스트림 수출(국가 간 이동)은 금지하는 협정서는?"

pred_data = create_squad_examples_from_arg(pred_question, pred_context)
print(pred_data.context_token_to_char)
print(pred_data.context_token_to_char[88])
print(pred_data.context_token_to_char[92])
pred_answer = predict_test(korquad_model, pred_data)
print(pred_answer)

['[CLS]', '정', '##부가', '이', '##산', '##화', '##탄', '##소', '스', '##트', '##림', '##의', '수', '##출', '##을', '가', '##능', '##하게', '하는', '런', '##던', '##의', '##정', '##서', '##개', '##정', '(', '2009년', '개', '##정', ')', '수', '##락', '##서를', '국제', '##해', '##사', '##기', '##구', '(', 'IM', '##O', ')', '사', '##무', '##국', '##에', '기', '##탁', '##할', '예', '##정이', '##라고', '29일', '밝혔다', '.', '이', '##산', '##화', '##탄', '##소', '스', '##트', '##림', '##은', '제', '##철', '##소', '##나', '발', '##전', '##소', '등에', '##서', '포', '##집', '##한', '이', '##산', '##화', '##탄', '##소', '##다', '.', '1996년', '채', '##택', '##된', '런', '##던', '##의', '##정', '##서는', '자', '##국', '해', '##역', '##에서', '이', '##산', '##화', '##탄', '##소', '스', '##트', '##림', '격', '##리', '(', '저', '##장', ')', '는', '허', '##용', '##하는', '반', '##면', ',', '이', '##산', '##화', '##탄', '##소', '스', '##트', '##림', '수', '##출', '(', '국가', '간', '이', '##동', ')', '은', '금', '##지', '##했다', '.', '2009년', '당', '##사', '##국', '##총', '##회', '##에서', '일', '##정', '절', '##차', '##에', '따라', '이', '##산', '##화

In [142]:
idx = 0
print(eval_squad_examples[idx].context)
print(eval_squad_examples[idx].question)
print(eval_squad_examples[idx].answer_text)   
print(eval_squad_examples[idx].start_char_idx, eval_squad_examples[idx].start_char_idx+len(eval_squad_examples[idx].answer_text))   

pred_context = eval_squad_examples[idx].context
pred_question = eval_squad_examples[idx].question

pred_data = create_squad_examples_from_arg(pred_question, pred_context)
pred_answer = predict_test(korquad_model, pred_data)
print(pred_answer)

1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다.
임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?
1989년 2월 15일
0 12
['[CLS]', '임', '##종', '##석', '##이', '여', '##의', '##도', '농', '##민', '폭', '##력', '시', '##위를', '주', '##도', '##한', '혐', '##의', '##로', '지', '##명', '##수', '##배', '된', '날', '##은', '?', '[SEP]']
padding_length= 193
1 3
(0, 5) (9, 12)
1989년 2월 15일


In [190]:
pred_context = """손흥민(토트넘 홋스퍼)으로부터 '세리머니 선물'을 받았던 토트넘 꼬마 팬이 '북런던 더비'에서 깜짝 페널티킥(PK)을 선보였다.
영국 매체 '풋볼 런던'은 12일(현지시간) 토트넘과 아스날 간의 잉글랜드 프리미어리그(EPL) 북런던 더비가 끝난 뒤 '어린 토트넘 팬 라일리 키스(Ryley Keys)가 PK를 기록했다'라는 제목의 기사를 보도했다.
이날 토트넘은 아스날과의 홈 경기에서 3대 0 승리를 거뒀다. 손흥민은 이번 경기에서 선발로 출전해 PK 유도, 상대 선수 퇴장 유도, 팀의 세 번째 득점 기록 등의 맹활약을 펼친 뒤 후반 72분 교체됐다.
토트넘의 꼬마 팬 라일리는 이날 토트넘이 2대 0으로 리드를 잡은 전반전이 끝난 뒤 하프 타임에 모습을 드러냈다."""
# pred_question = "라일리가 모습을 드러냈을때 스코어는?"
# pred_question = "토트넘과 아스널의 최종 스코어는?"
pred_question = "손흥민은 누구에게 깜짝 선물을 했는가?"
pred_data = create_squad_examples_from_arg(pred_question, pred_context)
pred_answer = predict_test(korquad_model, pred_data)
print(pred_answer)

padding_length= 249
26 18
(32, 33) (21, 22)

